In [ ]:
############################################################################
###### Ordering Manufacture Materials using BERT: 17/03/23, @Soumya D.######
############################################################################

In [ ]:
#Importing used packags

import numpy as np 
import pandas as pd 
import random as rn
import re
import nltk

import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from functools import partial

from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from nltk.stem import SnowballStemmer
from bertopic import BERTopic




#To remove warninigs
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Loadinig the data

data_raw = pd.read_excel(r'../data/data_sample.xlsx')

In [ ]:
# size of dataframe
print(data_raw.shape)
# display the head of data
display(data_raw.head())

In [ ]:
#details datafarme information
data_raw.info()

In [ ]:
#Copying a working df

data = data_raw

In [ ]:
data['Name'] = data['Name'].astype(str)

In [ ]:
txt = ' '.join(txt for txt in data.Name )
plt.figure(figsize=(15, 8))
wordcloud = WordCloud(
    background_color='black',
    max_font_size=100,
    max_words=100, 
    width=1000, 
    height=700
).generate(txt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### Basic Text Cleaninig 

In [ ]:
#function to clan punctuation

def clean_punctuation(text):
    result = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"',',') )
    return result

In [ ]:
# function to clean unify whitespaces

def clean_unify_whitespaces(text):
    cleaned_string = re.sub(' +', ' ', text )
    return cleaned_string

In [ ]:
#Clean Numeric

def clean_num(texts):
    output = re.sub(r'\d+', '', texts )
    return output

In [ ]:
#Basic characters cleaninig 

def basic_clean(txt):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', txt)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

In [ ]:
#Stop word cleaning 

stop = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

def clean_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop ]
    return ' '.join(text)

In [ ]:
#function to use stemming to normalize words

def Stemming(text):
    stem = []
    stopword = stopwords.words('english')
    snowball_stemmer = SnowballStemmer('english')
    word_tokens = nltk.word_tokenize(text)
    stemmed_word = [ snowball_stemmer.stem(word) for word in word_tokens ]
    stem = ' '.join(stemmed_word)
    return stem

#### Applying all these above methods to data[ 'Name' ]

In [ ]:
def cleaning(data, Name):
    data_cleaned  = data.copy()
    data_cleaned[Name] = data_cleaned['Name']
    data_cleaned[Name] = data_cleaned[Name].apply(clean_punctuation)
    data_cleaned[Name] = data_cleaned[Name].apply(clean_unify_whitespaces)
    data_cleaned[Name] = data_cleaned[Name].apply(clean_num)
    data_cleaned[Name] = data_cleaned[Name].apply(basic_clean)
    data_cleaned[Name] = data_cleaned[Name].apply(clean_stopwords)
    data_cleaned[Name] = data_cleaned[Name].apply(Stemming)
    return data_cleaned

data_cleaned = cleaning(data, 'clean_name' )

In [ ]:
data_cleaned.head(5)

In [ ]:
data_cleaned.columns.values.tolist()

In [ ]:
df = data_cleaned[['Name','clean_name','AT_MaraMatkl','AT_MaraMaktx','AT_MaraBrgew','AT_MaraMtart','AT_MaraLabor','AT_MaraBrgew_UnitID']] 

In [ ]:
df


#### BERT Topics and Document embedings 

In [ ]:
from bertopic import BERTopic

In [ ]:
#Create Topics

model = BERTopic(language="english")
topics, probs = model.fit_transform(df['clean_name'])

In [ ]:
model.get_topic(0)[:10]

In [ ]:
from bertopic.backend import languages
print(languages)

In [ ]:
###Embedding model

#st_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

In [ ]:
#Visualize Topics
model.visualize_topics()

In [ ]:
#Topic Reduction
model.reduce_topics(docs, nr_topics=60)

In [ ]:
#Topic Representation

model.update_topics(docs, n_gram_range=(1, 3))

In [ ]:
#Search Topics
similar_topics, similarity = model.find_topics("gpu", top_n=5); similar_topics

In [ ]:
model.get_topic(18)

In [ ]:
#Model serialization
# Save model
model.save("my_model")

# Load model
my_model = BERTopic.load("my_model")
